In [1]:
from abc import ABCMeta, abstractmethod
class Layer(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def forward(self, inputs):
        raise NotImplementedError()

    @abstractmethod
    def backward(self, inputs, layer_err):
        raise NotImplementedError()

    @abstractmethod
    def get_grad(self, inputs, layer_err):
        raise NotImplementedError()

    @abstractmethod
    def update(self, inputs, layer_err, lr):
        raise NotImplementedError()
    
    

In [2]:
import numpy as np
class ConvolutionalLayer(Layer):
    def __init__(self, filters, strides):
        self.filters = filters
        self.strides = strides
        
    def forward( self, inputs ):
        h_filter, w_filter, d1, d2 = self.filters.shape
        batch_size, h, w, d1 = inputs.shape

        output = np.zeros((batch_size, int( (h - h_filter) / self.strides[0] ) + 1 , int( (w - w_filter) / self.strides[1] ) + 1 , d2))

        for i in range(0, h - h_filter + 1,self.strides[0]):
            for j in range(0, w - w_filter + 1, self.strides[1]):
                output[:,i // self.strides[0],  j // self.strides[1], : ] = np.sum(inputs[:, i :i + h_filter, j : j + w_filter, :, np.newaxis] * self.filters, axis=(1, 2, 3))

        return output

    def backward(self, inputs, layer_err):
        h_filter, w_filter, d1, d2 = self.filters.shape
        batch_size, h, w, d2 = layer_err.shape

        d_output_h = (h - 1) * self.strides[0] + h_filter
        d_output_w = (w - 1) * self.strides[1] + w_filter

        d_output = np.zeros((batch_size, d_output_h, d_output_w, d1))

        for i in range(0 , d_output_h - h_filter + 1, self.strides[0]): 
            for j in range(0, d_output_w - w_filter+ 1, self.strides[1]):
                d_output[:, i:i + h_filter, j:j + w_filter, :] += np.sum(self.filters[np.newaxis, ...] * layer_err[:, int(i / self.strides[0]) : int(i / self.strides[0]) + 1, int(j / self.strides[1]) : int(j / self.strides[1]) + 1, np.newaxis, :], axis=4)
        return d_output

    def get_grad(self, inputs, layer_err):
                
        total_layer_err = np.sum(layer_err, axis=(0, 1, 2))
        filters_err = np.zeros(self.filters.shape)

        s1 = (inputs.shape[1] - self.filters.shape[0]) // self.strides[0] + 1
        s2 = (inputs.shape[2] - self.filters.shape[1]) // self.strides[1] + 1

        summed_inputs = np.sum(inputs, axis=0)

        for i in range(s1):
            for j in range(s2):
                filters_err += summed_inputs[i  * self.strides[0]:i * self.strides[0] + self.filters.shape[0], j * self.strides[1]:j * self.strides[1] + self.filters.shape[1], :, np.newaxis]
        return filters_err * total_layer_err
    def update(self, inputs, layer_err, lr):
        self.filters -= self.get_grad(inputs, layer_err) * lr
    
    

In [3]:
class MaxPool(Layer):
    
    def __init__(self, ksizes, strides):
        self.ksizes = ksizes
        self.strides = strides
        
    def forward(self,  inputs ):
        batch_size, h, w, d = inputs.shape

        output = np.zeros((batch_size, int( (h - self.ksizes[0]) / self.strides[0] ) + 1,int( (w - self.ksizes[1]) / self.strides[1] ) + 1 , d))

        for i in range(0, h - self.ksizes[0] + 1, self.strides[0]):
            for j in range(0, w - self.ksizes[1] + 1, self.strides[1]):
                output[:,i // self.strides[0],  j // self.strides[1], : ] = np.amax(inputs[:, i :i + self.ksizes[0], j : j + self.ksizes[1], :] , axis=(1, 2))

        return output

    def to_one_hat(self, y, d):
        o = np.zeros((len(y), self.ksizes[0], self.ksizes[1], d))
        for i in range(len(y)):
            o[i, y[i] // (self.ksizes[1] * d), (y[i]//d) % self.ksizes[1], y % d] = 1
        return o
    
    def backward(self, inputs, layer_err):
        batch_size, h2, w2, d = layer_err.shape

        h = (h2 -1) * self.strides[0] + self.ksizes[0]
        w = (w2 -1) * self.strides[1] + self.ksizes[1]

        output = np.zeros((batch_size, h, w , d))

        for i in range(0, h + 1 - self.ksizes[0], self.strides[0]):
            for j in range(0, w + 1 - self.ksizes[1], self.strides[1]):
                
                output[:, i:i + self.ksizes[0], j:j + self.ksizes[1], :] += layer_err[:, i // self.strides[0]:i // self.strides[0] + 1 , j // self.strides[1]:j // self.strides[1]+1 ,:] * self.to_one_hat(np.argmax(inputs[:, i :i + self.ksizes[0], j : j + self.ksizes[1], :].reshape(batch_size, -1) ,axis = 1), d)
                
        return output
    def update(self, inputs, layer_err, lr):
        pass
    

In [4]:
mpool = MaxPool((3,3), (2,2))
inputs = np.random.rand(50, 25,25 , 3)
# print(np.argmax(inputs, axis = 0).shape)
layer_err = np.random.rand(50, 12, 12, 3)
# print(mpool.forward(inputs))
mpool.backward(inputs, layer_err).shape

(50, 25, 25, 3)

In [5]:
class FullyConnected(Layer):
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def forward(self, inputs):
        return inputs.dot(self.w) + self.b

    def backward(self, inputs, layer_err):
        return layer_err.dot(self.w.T)

    def get_grad(self, inputs, layer_err):
        return inputs.T.dot(layer_err)
    
    def update(self, inputs, layer_err, lr):
        self.w -= self.get_grad(inputs, layer_err) * lr
        self.b -= np.sum(layer_err, axis = 0) * lr

In [6]:
class Flatten(Layer):

    def forward(self, inputs):
        return np.reshape(inputs, (inputs.shape[0], -1))

    def backward(self, inputs, layer_err):
        return np.reshape(layer_err, (inputs.shape))

    def get_grad(self, inputs, layer_err):
        return 0.

    def update(self, inputs, layer_err, lr):
        pass

In [7]:
class Activation(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def out(self,  x):
        pass

    @abstractmethod
    def derivetive(self, x):
        pass


In [8]:
class Relu(Activation):
    def out(self, x):
    #Rectified Linear Unit at x
        return (x+abs(x))/2.0
    def derivetive(self, x):
    #derivation of Rectified Linear Unit at x
        return (x>0)*1.0

class Sigmoid(Activation):
    def out(self, x):
    #sigmoid function at x (element wise)
    #input and output is a numpy matrix with size of [nx1]
        ### YOUR CODE HERE:
        return 1. / ( 1. + np.exp(-x))
        ### END YOUR CODE

    def derivetive(self, x):
    #derivation of sigmoid function at x
        tmp=self.out(x)
        return tmp * (1. - tmp)
    
class Identity(Activation):
    def out(self, x):
        return x
    def derivetive(self, x):
        return 1.0

def softmax_grad(s):
    out = np.diag(s)
    print(out.shape)

    for i in range(len(out)):
        for j in range(len(out)):
            if i == j:
                
                out[i][j] = s[i] * (1-s[i])
            else: 
                out[i][j] = -s[i]*s[j]
    return out

def softmax(x): 
#softmax function at x
#input and output is a numpy matrix with size of [nx1]
    ### YOUR CODE HERE:
    softmax_offset = np.max(x)
    o = np.exp(x - softmax_offset) / np.sum(np.exp(x - softmax_offset))
    return o
    ### END YOUR CODE


def softmax_ce_derivation(y_hat,y):
#derivation of CE(softmax(y),y_hat) respect to y
	return (softmax(y_hat)-y) / y_hat.shape[0]


def ce_error(y_hat_normalized,y):
#Cross Entropy error :CE(y,y_hat)
	return 	-1.0*np.multiply(y,np.log(y_hat_normalized+1.0e-20)).sum()

In [9]:
r = Relu()
print(r.derivetive(np.array([  1,-1, 0,0.00004])))


[ 1.  0.  0.  1.]


In [10]:
filters = np.random.rand(3, 3,3 , 1)
conv = ConvolutionalLayer(filters, (2,2))
inputs = np.random.rand(1, 25, 25, 3)

layer_err = np.random.rand(1, 12, 12, 1)
# print(conv.forward(inputs))
print(conv.backward( inputs, layer_err).shape)
conv.get_grad(inputs, layer_err)

(1, 25, 25, 3)


array([[[[ 5139.11107802],
         [ 5651.71394924],
         [ 5438.9726763 ]],

        [[ 5771.73652983],
         [ 5612.54261032],
         [ 5771.08384927]],

        [[ 5127.4364122 ],
         [ 5791.44669269],
         [ 5424.28880883]]],


       [[[ 4728.50124347],
         [ 5011.61884923],
         [ 5269.93092797]],

        [[ 5446.41691579],
         [ 5362.7878687 ],
         [ 5125.73773597]],

        [[ 4790.54577415],
         [ 5075.71883381],
         [ 5343.13744101]]],


       [[[ 5123.19774618],
         [ 5844.19437744],
         [ 5541.39379867]],

        [[ 5867.1362964 ],
         [ 5669.72743163],
         [ 5724.5418821 ]],

        [[ 5139.4665904 ],
         [ 6010.32094448],
         [ 5521.6493788 ]]]])

In [11]:
from tensorflow.examples.tutorials.mnist import input_data

#read data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
batch_size = 50
n_epochs = 100
learning_rate = 0.05

conv_filter_size = 4

conv_stride_size = 1
conv_layers = 64

pool_filter_size = 3
pool_stride_size = 2

fc1_size = 512

# filters = np.random.rand(conv_filter_size, conv_filter_size, 1, 3)
# conv = ConvolutionalLayer(filters, (conv_stride_size , conv_stride_size))
# mpool = MaxPool((pool_filter_size, pool_filter_size), (pool_stride_size,pool_stride_size))
# w = np.random.rand(144 * 3, 10)
# b = np.random.rand(10)
# fc = FullyConnected(w, b)
# layers_len = 4
# layers = [conv, mpool,Flatten(), fc]
# activations = [Sigmoid(), Sigmoid(), Identity(), Identity()]


w = np.random.normal(0,0.1,(784,10))
b = np.random.normal(0, 0.1, 10)

fc1 = FullyConnected(w, b)

w2 = np.random.normal(0,0.1,(10, 10))
b2 = np.random.normal(0, 0.1, 10)
fc2 = FullyConnected(w2, b2)

layers_len = 1
layers = [fc1, fc2]
activations = [Sigmoid(), Relu()]


## number of training batches
n_batches = int(mnist.train.num_examples / batch_size)

def train(X_batch, Y_batch):
    os = []
    zs = []
    os.append(X_batch)
    for i in range(layers_len):
        zs.append(layers[i].forward(os[i]))
        os.append(activations[i].out(zs[i]))

    dE_dO = softmax_ce_derivation(os[layers_len], Y_batch) 
    
    loss = ce_error(softmax(os[layers_len]), Y_batch)
    correct_preds = np.argmax(os[layers_len], axis = 1) == np.argmax(Y_batch, axis = 1)
    accuracy = np.sum(correct_preds) / float(len(X_batch))

    for i in range(layers_len - 1, -1, -1):
    
        dE_dz = dE_dO * activations[i].derivetive(zs[i])
        dE_dO = layers[i].backward(os[i], dE_dz)
        layers[i].update(os[i], dE_dz, learning_rate)

        
    return loss, accuracy
for i in range(100):  # train the model n_epochs times
    total_loss = 0
    total_acc = 0
    
    for _ in range(n_batches):
        ##training batches
        X_batch, Y_batch = mnist.train.next_batch(batch_size)

#         X_batch = X_batch.reshape(batch_size,1, 28, 28)
#         X_batch = np.transpose(X_batch, [0, 2, 3, 1])
        
        loss, acc = train(X_batch, Y_batch)    
        total_loss += loss
        total_acc += acc
#         print(acc)
#         print(loss)
    
    print('with epoch {}, Average loss : {}, Accuracy : {:.6f}'.format(i, total_loss / n_batches, total_acc / n_batches))



with epoch 0, Average loss : 309.30990369261406, Accuracy : 0.413527
with epoch 1, Average loss : 309.99359259267527, Accuracy : 0.507709
with epoch 2, Average loss : 310.1712651087862, Accuracy : 0.523164
with epoch 3, Average loss : 310.2647350356926, Accuracy : 0.534309
with epoch 4, Average loss : 310.32151049134393, Accuracy : 0.537727
with epoch 5, Average loss : 310.3636285708555, Accuracy : 0.544873
with epoch 6, Average loss : 310.3927588706572, Accuracy : 0.545400
with epoch 7, Average loss : 310.41486517635224, Accuracy : 0.544145
with epoch 8, Average loss : 310.436417058474, Accuracy : 0.544364
with epoch 9, Average loss : 310.45067154513856, Accuracy : 0.547818
with epoch 10, Average loss : 310.46654507438734, Accuracy : 0.545655
with epoch 11, Average loss : 310.4751476648913, Accuracy : 0.551964
with epoch 12, Average loss : 310.48838656879246, Accuracy : 0.548400
with epoch 13, Average loss : 310.49409435813, Accuracy : 0.548582
with epoch 14, Average loss : 310.503843

KeyboardInterrupt: 